# Creditos

Los datos de la manianera los obtuvimos del grupo @nostrodata
* GitHub : https://github.com/NOSTRODATA/conferencias_matutinas_amlo

# Importaciones
* Importamos la librería pandas para manipular fácilmente los archivos <code>.csv</code>.
* Importamos la librería glob para leer fácilmente los archivos <code>.csv</code>.

In [20]:
import pandas as pd
import glob
import re
import random as rd

# Limpieza de los datos
En la siguiente linea de código se usa la función <code>glob</code> para generalizar las carpetas del dataset para extraer todos los <code>.csv</code> de el mes de enero.


In [6]:
csv_files = glob.glob('conferenciasMatutinasAmlo/2022/1-2022/enero */mananera_*_01_2022.csv')
# print(csv_files)

Ahora que tenemos las rutas, iteramos sobre ellas y guardamos todos los datos en una lista llamada <code>filelist</code>.


In [7]:
filelist = []
for file in csv_files:
    df = pd.read_csv(file)
    filelist.append(df)

In [8]:
# Numero de manianeras de enero 2022
len(filelist)

17

Observamos uno de los dataframes de la lista.

In [9]:
filelist[2]

,Participante,Texto,Sentimiento,Palabras,Dia,Mes,Anio
0,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,Buenos días.,0.0,2,18,1,2022
1,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,"Pues vamos a informar, como todos los martes, ...",0.0,52,18,1,2022
2,JORGE ALCOCER VARELA,"Con su permiso, señor presidente.",0.0,5,18,1,2022
3,JORGE ALCOCER VARELA,Muy buenos días a todas y a todos ustedes.,0.0,9,18,1,2022
4,JORGE ALCOCER VARELA,"Como señala el señor presidente, en este momen...",0.0,55,18,1,2022
...,...,...,...,...,...,...,...
353,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,Todo esto lo comento porque sí hay toda una ca...,0.0,46,18,1,2022
354,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,Nada más porque por respeto no hablo de que en...,0.0,41,18,1,2022
355,PREGUNTA,¿Contra Del Mazo?,0.0,3,18,1,2022
356,PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR,"Y contra otras candidatas, que me enteré que ...",0.0,49,18,1,2022


Ahora limpiamos columas que no nos sirven mediante la función <code>drop()</code> de pandas

Después filtramos los participantes para sólo tener diálogos del presidente.

In [10]:
filteredDf = []
for df in filelist:
    # Eliminamos columnas
    df = df.drop(['Sentimiento', 'Palabras', 'Dia', 'Mes', 'Anio'], axis=1)

    # Filtramos para sólo tener participaciones del presidente
    df = df[df['Participante'] == 'PRESIDENTE ANDRES MANUEL LOPEZ OBRADOR']

    df = df.drop(['Participante'], axis=1)

    # left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
    
    # filteredDf.append(left_aligned_df)
    filteredDf.append(df)


Observamos uno de los dataframes para confirmar que las columnas se eliminaron y sólo hay participaciones del presidente.


In [11]:
filteredDf[0]

,Texto
0,Buenos días.
1,Vamos a iniciar la semana con el quién es quié...
44,Vamos a los videos.
98,Muy bien. Pues vamos adelante. Quedó pendient...
103,"Pues muy bien, hay que esperar a que resuelva..."
...,...
309,"Bueno, vamos a desayunar."
311,Con Beatriz.
313,"Al rato, al rato."
315,"Ah, pero ya tenemos el informe, te lo van a e..."


# Archivo de texto

La siguiente línea de código nos permite ver un ancho de las columnas determinado. Así podemos evadir que el dataframe nos dé los textos incompletos.

In [28]:
# Evitamos que se trunque
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)

Ahora que tenemos todos los datos filtrados, sólo falta guardar todo este contenido en un archivo de texto.


In [217]:
# with open("conferencias_matutinas_amlo.txt", "w", encoding='utf-8') as f:
#     for i, df in enumerate(filteredDf):
#         # print(type(df['Texto'][i]))
#         texto_str = df['Texto'].to_string(index=False) # Convert Series to string
#         f.write(texto_str + '\n')

Ahora que tenemos todos los datos filtrados, los guardaremos en un archivo, alineandolos a la izquierda y que cada oración dicha por AMLO esté en una línea.

In [12]:
with open("conferencias_matutinas_amlo.txt", "w", encoding='utf-8') as f:
    
    for i, df in enumerate(filteredDf):
        # print(type(df['Texto'][i]))

        # Cambiamos la indexacion para que este corrida
        inx = range(0,df.shape[0])
        dfIndex = df.reindex(index=inx) 

        for j in range(df.shape[0]) :

            texto_str = str(dfIndex['Texto'][j]).strip()

            if texto_str == 'nan' :
                continue

            # f.write(noPuntuacion(texto_str) + '\n')
            f.write(texto_str + '\n')


# Libreta 2

Tokenizaremos el texto. La tokezinación será por oración que dijo el presidente, y cada oración estará tokenizada por palabra. Quitaremos los signos de puntuación, pero dejaremos los acentos.

Función para quitar los signos de puntación y poner todo en minúsculas, además de poner el texto en una tuple.

In [13]:
def noPuntuacion(text): return re.findall(r'[a-zA-ZñÑáéíóúÁÉÍÓÚüÜ]+', text.lower())

In [14]:
texto = []
with open("conferencias_matutinas_amlo.txt", "r", encoding='utf-8') as f:

    for i,line in enumerate(f) :

        texto.append(noPuntuacion(line))


Partimos el corpus en dos de forma aleatoria, una parte para entrenamiento y la otra para prueba.

In [91]:
def separacionCorpus(text, porcentage, seed) :

    rd.seed(seed)


    textTrain = text.copy()
    textTest = []
    testPart = int(len(textTrain)*porcentage/100)

    for i in range(testPart) :

        index = rd.randint(0,len(textTrain)-1) 
        textTest.append(textTrain[index])
        del textTrain[index]

    return textTrain, textTest

# Pruebas

In [74]:

texto1 = [['buenos', 'dias','a','todos'],
                ['no','importa','que','digan'],
                ['niegalo','todo','oiste','niegalo'],
                ['todo'],
                ['niegalo1','todo1','oiste1','niegalo1'],
                ['niegalo2','todo2','oiste2','niegalo2'],
                ['niegalo3','todo3','oiste3','niegalo3'],
                ['niegalo4','todo4','oiste4','niegalo4'],
                ['niegalo5','todo5','oiste5','niegalo5'],
                ['niegalo6','todo6','oiste6','niegalo6']]

In [94]:
textoEnt, textoTes = separacionCorpus(texto1, 20, 16)

print('----------Texto original-----------------')
print(texto1)
print(len(texto1))
print('----------Entrenamiento-----------------')
print(textoEnt)
print(len(textoEnt))
print('-----------Prueba----------------')
print(textoTes)
print(len(textoTes))

----------Texto original-----------------
[['buenos', 'dias', 'a', 'todos'], ['no', 'importa', 'que', 'digan'], ['niegalo', 'todo', 'oiste', 'niegalo'], ['todo'], ['niegalo1', 'todo1', 'oiste1', 'niegalo1'], ['niegalo2', 'todo2', 'oiste2', 'niegalo2'], ['niegalo3', 'todo3', 'oiste3', 'niegalo3'], ['niegalo4', 'todo4', 'oiste4', 'niegalo4'], ['niegalo5', 'todo5', 'oiste5', 'niegalo5'], ['niegalo6', 'todo6', 'oiste6', 'niegalo6']]
10
----------Entrenamiento-----------------
[['buenos', 'dias', 'a', 'todos'], ['no', 'importa', 'que', 'digan'], ['niegalo', 'todo', 'oiste', 'niegalo'], ['todo'], ['niegalo1', 'todo1', 'oiste1', 'niegalo1'], ['niegalo3', 'todo3', 'oiste3', 'niegalo3'], ['niegalo4', 'todo4', 'oiste4', 'niegalo4'], ['niegalo6', 'todo6', 'oiste6', 'niegalo6']]
8
-----------Prueba----------------
[['niegalo2', 'todo2', 'oiste2', 'niegalo2'], ['niegalo5', 'todo5', 'oiste5', 'niegalo5']]
2


# ------------------------------------------------------